In [2]:
import pandas as pd
import os
import glob
import numpy as np
import pandas_ta as ta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
#info from https://data.binance.vision/?prefix=data/spot/monthly/klines/BTCUSDT/5m/
#backtesting this idea from youtube https://www.youtube.com/watch?v=ydolTWrM5vc

C:\Users\Jordi\anaconda3\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [3]:
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "input/*.csv"))

In [4]:
df_all = pd.DataFrame()
for f in csv_files:
    df = pd.read_csv(f, names=['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'])
    df_all = pd.concat([df,df_all])

In [5]:
df_all = df_all.sort_values('Open time', ascending=True)

In [6]:
df_all['HA_Open'] = (df_all.Open + df_all.shift(1).Close) * 1/2

In [7]:
df_all['HA_Close'] = (df_all.Open + df_all.Close + df_all.High + df_all.Low) * 1/4

In [8]:
df_all['HA_High'] = df_all[['Open', 'Close', 'High']].max(axis=1)

In [9]:
df_all['HA_Min'] = df_all[['Open', 'Close', 'High']].min(axis=1)

In [10]:
df_all['EMA_200'] = df_all.ta.ema(length=200)

In [11]:
df_all['EMA_50'] = df_all.ta.ema(length=50)

In [12]:
df_all = pd.concat([df_all,df_all.ta.adx()], axis=1)

In [13]:
df_all['Volume_20MA'] = df_all.Volume.rolling(window=20).mean()

In [14]:
df_all['date'] = pd.to_datetime(df_all['Open time'], unit='ms', origin='unix')

In [15]:
df_all = df_all.set_index('date')

In [16]:
df_all['HA_color'] = df_all['HA_Close'] > df_all['HA_Open']

In [17]:
df_all.head()

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,...,HA_Close,HA_High,HA_Min,EMA_200,EMA_50,ADX_14,DMP_14,DMN_14,Volume_20MA,HA_color
date,,,,,,,,,,,,,,,,,,,,,
2019-12-01 00:00:00,1575158400000,7540.63,7541.85,7530.00,7532.23,84.165204,1575158699999,6.342718e+05,801,30.139058,...,7536.1775,7541.85,7532.23,NaN,NaN,NaN,NaN,NaN,NaN,False
2019-12-01 00:05:00,1575158700000,7530.85,7533.51,7519.00,7524.46,182.159347,1575158999999,1.371344e+06,1239,92.656946,...,7526.9550,7533.51,7524.46,NaN,NaN,NaN,NaN,NaN,NaN,False
2019-12-01 00:10:00,1575159000000,7523.71,7524.73,7493.01,7504.25,263.774584,1575159299999,1.980126e+06,1782,126.568561,...,7511.4250,7524.73,7504.25,NaN,NaN,NaN,NaN,NaN,NaN,False
2019-12-01 00:15:00,1575159300000,7506.99,7514.71,7495.04,7496.00,154.470096,1575159599999,1.159193e+06,1101,83.957069,...,7503.1850,7514.71,7496.00,NaN,NaN,NaN,NaN,NaN,NaN,False
2019-12-01 00:20:00,1575159600000,7496.00,7501.79,7483.24,7500.42,164.145579,1575159899999,1.229991e+06,1255,101.702759,...,7495.3625,7501.79,7496.00,NaN,NaN,NaN,NaN,NaN,NaN,False


In [18]:
df_all['HA_color'] = np.where(df_all['HA_color'], 1, 0)

In [19]:
df_all['HA_color']

date
2019-12-01 00:00:00    0
2019-12-01 00:05:00    0
2019-12-01 00:10:00    0
2019-12-01 00:15:00    0
2019-12-01 00:20:00    0
                      ..
2022-05-31 23:35:00    0
2022-05-31 23:40:00    0
2022-05-31 23:45:00    1
2022-05-31 23:50:00    0
2022-05-31 23:55:00    0
Name: HA_color, Length: 262498, dtype: int32

In [20]:
df_all['Signal_long'] = np.where(((df_all.EMA_50 > df_all.EMA_200) & (df_all.HA_color == 1) & (df_all.Volume > df_all.Volume.shift(1)) & (df_all.ADX_14 > 25)), 1, 0)

In [21]:
df_all['Signal_short'] = np.where(((df_all.EMA_50 < df_all.EMA_200) & (df_all.HA_color == 0) & (df_all.Volume > df_all.Volume.shift(1)) & (df_all.ADX_14 > 25)), 2, 0)

In [22]:
# 1 is long signal
# 2 is short signal
# (Not implemented)
# 3 is sell signal (for long trades)
# 4 is "sell" signal (for short trades)

In [23]:
df_all['Signal'] = df_all[['Signal_long', 'Signal_short']].max(axis=1)

In [24]:
def get_candle_color():
    return df_all[250:].HA_color

In [25]:
def get_signal():
    return df_all[250:].Signal

In [26]:
def get_HA_min():
    return df_all[250:].HA_Min

In [27]:
def get_HA_high():
    return df_all[250:].HA_High

# not working
from backtesting import Strategy


class strat1(Strategy):
    def init(self):
        super().init()
        self.signal1 = self.I(get_signal)
        self.candle = self.I(get_candle_color)
        self.ha_min = self.I(get_HA_min)
        self.ha_high = self.I(get_HA_high)

    def next(self):
        super().next() 
        if self.signal1==1:
            sl1 = self.ha_min[-1]
            #tp1 = self.data.ha_close[-1] + 600e-4
            self.buy(sl=sl1) #, tp=tp1)
            
        if self.candle == 0:
            self.sell()
            
        if self.signal1==2:
            sl1 = self.ha_high[-1]
            #tp1 = self.data.Close[-1] - 600e-4
            self.sell(sl=sl1) #, tp=tp1)
            
        if self.candle == 1:
            self.buy()

In [28]:
class strat1_long(Strategy):
    def init(self):
        #super().init()
        self.signal1 = self.I(get_signal)
        self.candle = self.I(get_candle_color)
        self.ha_min = self.I(get_HA_min)
        self.ha_high = self.I(get_HA_high)

    def next(self):
        #super().next() 
        if self.signal1==1:
            sl1 = self.ha_min[-1]
            #tp1 = self.data.ha_close[-1] + 600e-4
            self.buy(sl=sl1) #, tp=tp1)
        if self.candle == 0:
            self.sell()
        if self.signal1==2:
            sl1 = self.ha_high[-1]
            #tp1 = self.data.Close[-1] - 600e-4
            self.sell(sl=sl1) #, tp=tp1)
        if self.candle == 1:
            self.buy()

In [29]:
class strat1_short(Strategy):
    def init(self):
        #super().init()
        self.signal1 = self.I(get_signal)
        self.candle = self.I(get_candle_color)
        self.ha_min = self.I(get_HA_min)
        self.ha_high = self.I(get_HA_high)

    def next(self):
        #super().next() 
        if self.signal1==2:
            sl1 = self.ha_high[-1]
            #tp1 = self.data.Close[-1] - 600e-4
            self.sell(sl=sl1) #, tp=tp1)
            
        if self.candle == 1:
            self.buy()

In [30]:
bt_l = Backtest(df_all[250:], strat1_long, cash=100_000, commission=.002, exclusive_orders=True, trade_on_close=True)
stat_l = bt_l.run()
stat_l

Start                     2019-12-01 20:50:00
End                       2022-05-31 23:55:00
Duration                    912 days 03:05:00
Exposure Time [%]                     0.65358
Equity Final [$]                      4516.89
Equity Peak [$]                        100133
Return [%]                           -95.4831
Buy & Hold Return [%]                 331.866
Return (Ann.) [%]                    -70.0483
Volatility (Ann.) [%]                 18.8955
Sharpe Ratio                                0
Sortino Ratio                               0
Calmar Ratio                                0
Max. Drawdown [%]                    -95.4891
Avg. Drawdown [%]                    -95.4891
Max. Drawdown Duration      912 days 02:55:00
Avg. Drawdown Duration      912 days 02:55:00
# Trades                                 1670
Win Rate [%]                          10.1198
Best Trade [%]                        18.1925
Worst Trade [%]                      -7.68774
Avg. Trade [%]                    

In [31]:
stat_l._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-13,1,2,7336.19820,7326.00,132.57660,0.001390,2019-12-01 20:55:00,2019-12-01 21:00:00,0 days 00:05:00
1,-13,2,3,7311.34800,7328.16,-218.55600,-0.002299,2019-12-01 21:00:00,2019-12-01 21:05:00,0 days 00:05:00
2,13,3,4,7342.81632,7312.64,-392.29216,-0.004110,2019-12-01 21:05:00,2019-12-01 21:10:00,0 days 00:05:00
3,-13,4,5,7298.01472,7346.16,-625.88864,-0.006597,2019-12-01 21:10:00,2019-12-01 21:15:00,0 days 00:05:00
4,13,5,6,7360.85232,7342.64,-236.76016,-0.002474,2019-12-01 21:15:00,2019-12-01 21:20:00,0 days 00:05:00
...,...,...,...,...,...,...,...,...,...,...
1665,-1,30298,30299,4520.25138,4559.30,-39.04862,-0.008639,2020-03-16 10:35:00,2020-03-16 10:40:00,0 days 00:05:00
1666,-1,30300,30301,4535.63056,4539.73,-4.09944,-0.000904,2020-03-16 10:45:00,2020-03-16 10:50:00,0 days 00:05:00
1667,-1,30301,30302,4530.65054,4450.95,79.70054,0.017591,2020-03-16 10:50:00,2020-03-16 10:55:00,0 days 00:05:00
1668,-1,30302,30303,4442.04810,4547.53,-105.48190,-0.023746,2020-03-16 10:55:00,2020-03-16 11:00:00,0 days 00:05:00


In [32]:
bt_s = Backtest(df_all[250:], strat1_short, cash=100_000, commission=.002, exclusive_orders=True)
stat_s = bt_s.run()
stat_s

Start                     2019-12-01 20:50:00
End                       2022-05-31 23:55:00
Duration                    912 days 03:05:00
Exposure Time [%]                     1.21984
Equity Final [$]                      4506.32
Equity Peak [$]                        100044
Return [%]                           -95.4937
Buy & Hold Return [%]                 331.866
Return (Ann.) [%]                    -70.6808
Volatility (Ann.) [%]                 14.8163
Sharpe Ratio                                0
Sortino Ratio                               0
Calmar Ratio                                0
Max. Drawdown [%]                    -95.4957
Avg. Drawdown [%]                    -47.9438
Max. Drawdown Duration      912 days 02:40:00
Avg. Drawdown Duration      456 days 01:25:00
# Trades                                 1769
Win Rate [%]                          10.6275
Best Trade [%]                        18.1922
Worst Trade [%]                      -8.26841
Avg. Trade [%]                    

In [33]:
#bt.run()
#bt.plot()